In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
!pip install efficientnet-pytorch

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image, ImageFile
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from efficientnet_pytorch import EfficientNet
from IPython.display import display, Javascript
from sklearn.model_selection import train_test_split

# 잘린 이미지 처리를 위해 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Colab 연결 유지 함수
def keep_alive():
    display(Javascript('''
        function ClickConnect(){
            console.log("클릭 연결 버튼");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 60000)
    '''))

In [ ]:
# 데이터셋 클래스 정의
# 이 클래스는 데이터를 불러오고 처리하는 방법을 정의
# img_dir은 이미지가 저장된 폴더 경로, df는 데이터프레임, facepart는 얼굴 부위 번호
# 데이터셋 클래스 정의
class CachedDataset(Dataset):
    def __init__(self, img_dir, df, facepart, transform=None, cache_dir='/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단'):
        self.transform = transform
        self.facepart = facepart
        os.makedirs(cache_dir, exist_ok=True)
        self.cache_file = os.path.join(cache_dir, f'cache_facepart_{facepart}.pkl')

        if os.path.exists(self.cache_file):
            print(f"facepart {facepart}의 캐시된 데이터를 불러옵니다...")
            with open(self.cache_file, 'rb') as f:
                self.cache = pickle.load(f)
        else:
            print(f"facepart {facepart}의 캐시를 생성합니다...")
            self.cache = []
            df_facepart = df[df['images'].apply(lambda x: x['facepart'] == facepart)]

            for _, row in df_facepart.iterrows():
                try:
                    img_name = row['info']['filename']
                    if facepart == 0:
                        img_path = os.path.join(img_dir, img_name)
                    else:
                        img_path = os.path.join(img_dir, f"{os.path.splitext(img_name)[0]}_{facepart}.jpg")

                    if not os.path.exists(img_path):
                        print(f"이미지를 찾을 수 없습니다: {img_path}")
                        continue

                    try:
                        image = Image.open(img_path).convert('RGB')
                    except (IOError, OSError):
                        print(f"손상된 이미지 파일입니다: {img_path}")
                        continue

                    if self.transform:
                        image = self.transform(image)

                    labels = self._prepare_labels(row['annotations'], row['equipment'], row['info'])
                    self.cache.append((image, torch.tensor(labels, dtype=torch.float)))
                except Exception as e:
                    print(f"데이터 처리 중 오류 발생: {str(e)}")
                    continue

            with open(self.cache_file, 'wb') as f:
                pickle.dump(self.cache, f)
            print(f"facepart {facepart}의 캐시가 생성되고 저장되었습니다")

    def _prepare_labels(self, annotations, equipment, info):
        labels = []
        try:
            if self.facepart == 0:
                labels = [info['skin_type'], info['sensitive']]
                if annotations:
                    labels.extend(self._process_annotations(annotations))
                if equipment:
                    labels.extend(list(equipment.values()))
            else:
                if annotations:
                    labels.extend(self._process_annotations(annotations))
                if equipment:
                    labels.extend(list(equipment.values()))
        except Exception as e:
            print(f"레이블 준비 중 오류 발생: {str(e)}")
        return labels

    def _process_annotations(self, annotations):
        processed = []
        for key, value in annotations.items():
            if key == 'acne' and isinstance(value, list):
                processed.append(len(value))  # 여드름 개수
            elif isinstance(value, (int, float)):
                processed.append(value)
        return processed

    def __len__(self):
        return len(self.cache)

    def __getitem__(self, idx):
        return self.cache[idx]

In [ ]:
# ResNet50 모델 생성 함수
def create_resnet_model(num_outputs):
    model = models.resnet50(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, num_outputs)
    )
    return model

In [ ]:
# EfficientNet-B0 모델 생성 함수
def create_efficientnet_model(num_outputs):
    model = EfficientNet.from_pretrained('efficientnet-b0')
    num_features = model._fc.in_features
    model._fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, num_outputs)
    )
    return model

In [ ]:
# 모델 학습 함수
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, facepart, model_name):
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    save_path = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단'

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)
        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

        # 검증
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        print(f"Validation Loss: {val_loss:.4f}")

        # 최고의 모델 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), os.path.join(save_path, f'best_model_{model_name}_facepart_{facepart}.pth'))
            print(f"facepart {facepart}의 새로운 최고 모델을 저장했습니다")

        # 체크포인트 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
        }, os.path.join(save_path, f'checkpoint_{model_name}_facepart_{facepart}_epoch_{epoch+1}.pth'))

    # 학습 과정 시각화 및 저장
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
    plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss for {model_name} Facepart {facepart}')
    plt.legend()
    plt.xticks(range(0, num_epochs+1, 5))
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f'loss_plot_{model_name}_facepart_{facepart}.png'))
    plt.close()

    return model

In [ ]:
# 메인 함수
def main():
    base_path = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터'

    try:
        df = pd.read_csv(os.path.join(base_path, 'json to df.csv'))
    except FileNotFoundError:
        print("CSV 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        return
    except pd.errors.EmptyDataError:
        print("CSV 파일이 비어있습니다.")
        return
    except pd.errors.ParserError:
        print("CSV 파일 파싱 중 오류가 발생했습니다. 파일 형식을 확인해주세요.")
        return

    # 문자열 딕셔너리를 실제 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

    # Training 데이터만 선택
    df_train = df[df['split'] == 'Training']

    # 이미지 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for facepart in range(9):
        print(f"facepart {facepart} 처리 중")

        if facepart == 0:
            img_dir = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training/01.원천데이터'
        else:
            img_dir = f'/gdrive/MyDrive/Final project/1_Red/4_데이터탐색_전처리/facepart별 피부 이미지/Training_cropped/{facepart}'

        if not os.path.exists(img_dir):
            print(f"이미지 디렉토리를 찾을 수 없습니다: {img_dir}")
            continue

        # 데이터셋 생성
        try:
            dataset = CachedDataset(
                img_dir,
                df_train,
                facepart,
                transform,
                cache_dir='/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단'
            )
        except Exception as e:
            print(f"데이터셋 생성 중 오류 발생: {str(e)}")
            continue

        # 데이터셋을 train과 validation으로 분할
        train_data, val_data = train_test_split(dataset, test_size=0.1, random_state=42)

        # 데이터 로더 생성
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=4)

        # 모델 생성
        num_outputs = len(dataset[0][1])
        resnet_model = create_resnet_model(num_outputs).to(device)
        efficientnet_model = create_efficientnet_model(num_outputs).to(device)

        # 손실 함수와 최적화 알고리즘 정의
        criterion = nn.MSELoss()
        resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
        efficientnet_optimizer = optim.Adam(efficientnet_model.parameters(), lr=0.001)

        # ResNet50 모델 학습
        try:
            resnet_model = train_model(resnet_model, train_loader, val_loader, criterion, resnet_optimizer, num_epochs=30, device=device, facepart=facepart, model_name='resnet50')
        except Exception as e:
            print(f"ResNet50 모델 학습 중 오류 발생: {str(e)}")

        # EfficientNet-B0 모델 학습
        try:
            efficientnet_model = train_model(efficientnet_model, train_loader, val_loader, criterion, efficientnet_optimizer, num_epochs=30, device=device, facepart=facepart, model_name='efficientnet_b0')
        except Exception as e:
            print(f"EfficientNet-B0 모델 학습 중 오류 발생: {str(e)}")

        # 최종 모델 저장
        save_path = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단'
        try:
            torch.save(resnet_model.state_dict(), os.path.join(save_path, f'final_model_resnet50_facepart_{facepart}.pth'))
            torch.save(efficientnet_model.state_dict(), os.path.join(save_path, f'final_model_efficientnet_b0_facepart_{facepart}.pth'))
        except Exception as e:
            print(f"모델 저장 중 오류 발생: {str(e)}")

In [ ]:
# 테스트용 facepart 7

# 메인 함수
def main():
    base_path = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터'

    try:
        df = pd.read_csv(os.path.join(base_path, 'json to df.csv'))
    except FileNotFoundError:
        print("CSV 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        return
    except pd.errors.EmptyDataError:
        print("CSV 파일이 비어있습니다.")
        return
    except pd.errors.ParserError:
        print("CSV 파일 파싱 중 오류가 발생했습니다. 파일 형식을 확인해주세요.")
        return

    # 문자열 딕셔너리를 실제 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

    # Training 데이터만 선택
    df_train = df[df['split'] == 'Training']

    # 이미지 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for facepart in range(7,8):
        print(f"facepart {facepart} 처리 중")

        if facepart == 0:
            img_dir = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training/01.원천데이터'
        else:
            img_dir = f'/gdrive/MyDrive/Final project/1_Red/4_데이터탐색_전처리/facepart별 피부 이미지/Training_cropped/{facepart}'

        if not os.path.exists(img_dir):
            print(f"이미지 디렉토리를 찾을 수 없습니다: {img_dir}")
            continue

        # 데이터셋 생성
        try:
            dataset = CachedDataset(
                img_dir,
                df_train,
                facepart,
                transform,
                cache_dir='/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단'
            )
        except Exception as e:
            print(f"데이터셋 생성 중 오류 발생: {str(e)}")
            continue

        # 데이터셋을 train과 validation으로 분할
        train_data, val_data = train_test_split(dataset, test_size=0.1, random_state=42)

        # 데이터 로더 생성
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=4)

        # 모델 생성
        num_outputs = len(dataset[0][1])
        resnet_model = create_resnet_model(num_outputs).to(device)
        efficientnet_model = create_efficientnet_model(num_outputs).to(device)

        # 손실 함수와 최적화 알고리즘 정의
        criterion = nn.MSELoss()
        resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
        efficientnet_optimizer = optim.Adam(efficientnet_model.parameters(), lr=0.001)

        # ResNet50 모델 학습
        try:
            resnet_model = train_model(resnet_model, train_loader, val_loader, criterion, resnet_optimizer, num_epochs=30, device=device, facepart=facepart, model_name='resnet50')
        except Exception as e:
            print(f"ResNet50 모델 학습 중 오류 발생: {str(e)}")

        # EfficientNet-B0 모델 학습
        try:
            efficientnet_model = train_model(efficientnet_model, train_loader, val_loader, criterion, efficientnet_optimizer, num_epochs=30, device=device, facepart=facepart, model_name='efficientnet_b0')
        except Exception as e:
            print(f"EfficientNet-B0 모델 학습 중 오류 발생: {str(e)}")

        # 최종 모델 저장
        save_path = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단'
        try:
            torch.save(resnet_model.state_dict(), os.path.join(save_path, f'final_model_resnet50_facepart_{facepart}.pth'))
            torch.save(efficientnet_model.state_dict(), os.path.join(save_path, f'final_model_efficientnet_b0_facepart_{facepart}.pth'))
        except Exception as e:
            print(f"모델 저장 중 오류 발생: {str(e)}")

In [ ]:
if __name__ == "__main__":
    keep_alive()  # 연결 유지 함수 실행
    main()

In [ ]:
# 성능 시각화 함수
import matplotlib.pyplot as plt

def plot_performance(results):
    metrics = ['box_loss', 'cls_loss', 'dfl_loss', 'mAP50', 'mAP50-95', 'precision', 'recall']

    fig, axs = plt.subplots(3, 3, figsize=(20, 20))
    axs = axs.ravel()  # 2D array를 1D array로 변환

    for i, metric in enumerate(metrics):
        ax = axs[i]
        if metric in results.results_dict:
            ax.plot(results.results_dict[metric], label=f'train_{metric}')
            if f'val_{metric}' in results.results_dict:
                ax.plot(results.results_dict[f'val_{metric}'], label=f'val_{metric}')
            ax.set_title(f'{metric.replace("_", " ").capitalize()}')
            ax.set_xlabel('Epoch')
            ax.set_ylabel('Value')
            ax.legend()
            ax.set_xticks(range(0, 31))

    # 사용하지 않는 서브플롯 제거
    for j in range(i+1, 9):
        fig.delaxes(axs[j])

    plt.tight_layout()
    plt.savefig('training_performance1.png')
    plt.close()

    print("성능 그래프가 'training_performance1.png'로 저장되었습니다.")